In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
from sklearn.linear_model import Lasso
from sklearn.preprocessing import normalize
 
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, cross_validate

In [2]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
#d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
#d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

In [3]:
y_class = outputs["dement_fail"].values
meaningful_idxs = np.where(y_class==1)
y = outputs["dement_time_age"].values[meaningful_idxs]
C = d_clinical.values[meaningful_idxs]
#G = d_genetic.values[meaningful_idxs]
#V = d_vampire.values[meaningful_idxs]

## Dementia

In [4]:
y = outputs["cvd_fail"].values
C = d_clinical.values
#G = d_genetic.values
#V = d_vampire.values

In [5]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y))

In [6]:
y_ = y[tr_idx]
y_test = y[ts_idx]
C_ = C[tr_idx]
C_test = C[ts_idx]
#G_ = G[tr_idx]
#G_test = G[ts_idx]
#V_ = V[tr_idx]
#V_test = V[ts_idx]

In [7]:
ds_list = [C_]#, G_, V_]
ds_test = [C_test]#, G_test, V_test]
ds_names = ['clinic']#, 'genetic', 'vampire']

In [8]:
def centering_normalizing(train, test, exclusion_list = None):
    
    if exclusion_list is not None:
        scale, train = ut.centering_normalizing(train, exclusion_list)
    else:
        scale, train = ut.centering_normalizing(train)

    new_Xts = test-scale

    if exclusion_list is not None:
        new_Xts[:, exclusion_list] = test[:, exclusion_list]

    return train, new_Xts, scale

def normalizing(train, test):
    return normalize(train), normalize(test)

In [9]:
def make_regression(X, y, sparsity):

    scores = np.zeros(len(sparsity))
    
    for idx, sp in enumerate(sparsity):
        model = Lasso(alpha = sp)
        scores[idx] = np.mean(cross_validate(model,  X, y, return_train_score=False, cv=3)['test_score'])
        
    return sparsity[np.argmax(scores)]

In [10]:
def learn(C_, C_test, y, y_test, sparsity, centering = False, norm = False):
    if centering == True:
        C_, C_test, _ = centering_normalizing(C_, C_test)#, [5,6,7,9,10,13,15,16,17,18,19])

    if norm == True:
         C_, C_test = normalizing(C_, C_test)

    best_alpha = make_regression(C_, y, sparsity)
    model = Lasso(alpha = best_alpha)
    model.fit(C_, y)
    y_pred = model.predict(C_test)
    coef = model.coef_
    
    meanErr = np.mean(np.abs(y_pred-y_test))
    varErr = np.var(np.abs(y_pred-y_test))

    print("Average error: {}".format(meanErr))
    print("Error variance: {}".format(varErr))
    print("Coef: {}".format(coef))
    print("BestAlpha: {}".format(best_alpha))

In [ ]:
sparsity = [0.01, 0.03, 0.05, 0.07, 0.1, 0.12]

## Basic approach

In [11]:
learn(C_, C_test, y_, y_test, sparsity)

Average error: 0.798072716451704
Error variance: 0.19173417225444245
Coef: [ 0.00302669  0.00628166  0.         -0.01126625  0.00344043  0.
 -0.         -0.          0.01209665  0.05631938 -0.01268256  0.04079596
 -0.04153195  0.24105243 -0.00664638 -0.01504809  0.          0.
  0.00668063 -0.          0.03210211]
BestAlpha: 0.01


C:\Users\Lorenzo\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


## Normalized data

In [12]:
learn(C_, C_test, y_, y_test, sparsity, norm = True)

Average error: 0.8839735268455662
Error variance: 0.10223714891207361
Coef: [-0.  0. -0. -0.  0.  0.  0. -0.  0.  0. -0.  0. -0.  0. -0. -0. -0.  0.
  0.  0.  0.]
BestAlpha: 0.01


## Origin Data Centering

In [13]:
learn(C_, C_test, y_, y_test, sparsity, centering = True)

Average error: 3.3485817029263245
Error variance: 6.033243528951659
Coef: [-0.          0.         -0.         -0.21727138  0.07391699  0.
 -0.         -0.          0.26572281  0.         -0.          0.
 -0.          1.20944333 -0.         -0.         -0.          0.
  0.          0.          0.        ]
BestAlpha: 0.01


## Origin Data  Centering and Normalization

In [14]:
learn(C_, C_test, y_, y_test, sparsity, centering = True, norm = True)

Average error: 0.8432866613431059
Error variance: 0.13348983190138347
Coef: [-0.          0.         -0.         -0.21727138  0.07391699  0.
 -0.         -0.          0.26572281  0.         -0.          0.
 -0.          1.20944333 -0.         -0.         -0.          0.
  0.          0.          0.        ]
BestAlpha: 0.01
